# We will download the data from the `scikit-fingerprints/MoleculeNet_ClinTox` hugging face site

In [1]:
import pandas as pd

In [3]:
import pandas as pd

df = pd.read_csv("hf://datasets/scikit-fingerprints/MoleculeNet_ClinTox/clintox.csv")

In [4]:
df.head()

,SMILES,FDA_APPROVED,CT_TOX
0,[C@@H]1([C@@H]([C@@H]([C@H]([C@@H]([C@@H]1Cl)C...,1,0
1,[C@H]([C@@H]([C@@H](C(=O)[O-])O)O)([C@H](C(=O)...,1,0
2,[H]/[NH+]=C(/C1=CC(=O)/C(=C\C=c2ccc(=C([NH3+])...,1,0
3,[H]/[NH+]=C(\N)/c1ccc(cc1)OCCCCCOc2ccc(cc2)/C(...,1,0
4,[N+](=O)([O-])[O-],1,0
